In [6]:
#import pylab
import sklearn
from sklearn.utils import Bunch
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
#import os
import nltk
#import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [7]:
comment_data = pd.read_csv('/Users/gregdecanio/Desktop/CAP4770/Group7_Project/CAP4770Group7/new_data.csv')

In [8]:
#comment_data.head()
comment_data

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,510625,0.000000,"What a great topic to focus on: ""how the Catho...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,147774,approved,0,0,0,0,0,0.000000,4,4
1,1004092,0.166667,There is so much of this program that is compl...,0.000000,0.000000,0.000000,0.166667,0.000000,NaN,NaN,...,168561,approved,0,0,0,2,3,0.000000,0,6
2,5175499,0.000000,"I'm sorry, but when you run as a Republican, s...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,329449,approved,0,0,0,11,2,0.000000,10,4
3,5903235,0.000000,Torytwo. Your names says it all.,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,373894,approved,0,0,0,0,0,0.000000,0,4
4,5884341,0.000000,I wouldn't brag about being a Bernie supporter...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,373006,approved,0,0,0,1,0,0.000000,0,4
5,557701,0.000000,"""I would have thought the Koch brothers' embra...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,149747,approved,0,0,0,2,0,0.000000,10,4
6,1055860,0.500000,Its painfully clear that Donald Trump's only c...,0.000000,0.000000,0.100000,0.500000,0.100000,NaN,NaN,...,315660,approved,2,0,0,33,13,0.000000,0,10
7,6088691,0.400000,"Metasphere writes:\n\n""Trudeau is a U.S. lapdo...",0.000000,0.000000,0.000000,0.400000,0.000000,NaN,NaN,...,385469,approved,0,0,1,12,7,0.000000,0,10
8,5036889,0.738462,Anti-Russian--you hypocrite!\nDo you call anyo...,0.046154,0.061538,0.215385,0.692308,0.061538,NaN,NaN,...,314105,approved,0,0,0,0,0,0.000000,0,65
9,310265,0.000000,"Gee, Nigel, another notch on your belt. Thank...",0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,75500,approved,0,0,0,1,0,0.000000,0,4


In [ ]:
#%matplotlib inline
#plot = comment_data['target'].hist(bins=20)
#print comment_data['target'].describe()

# Adding Discrete Classification to Data
### I will be classifying the data into 4 categories:
Not toxic: target < 0.5  
Toxic: target >= 0.5

In [9]:
%%time
def classifier(row):
  if row['target'] >= 0.5:
    return 'toxic'
  elif row['target'] < 0.5:
    return 'not-toxic'
  else:
    return 'undefined'

comment_data['toxicity_classification'] = comment_data.apply(classifier, axis=1)
comment_data.groupby(['toxicity_classification']).size().plot(kind='bar')
print comment_data.groupby(['toxicity_classification']).size()
print "Non-toxic count: ", comment_data.groupby(['toxicity_classification']).size()['not-toxic']
print "Toxic count: ", comment_data.groupby(['toxicity_classification']).size()['toxic']

toxicity_classification
not-toxic    96851
toxic        18149
dtype: int64
Non-toxic count:  96851
Toxic count:  18149
CPU times: user 4.77 s, sys: 333 ms, total: 5.1 s
Wall time: 6.82 s


# Cleaning Comment Text
## Removing Stop Words
https://stackoverflow.com/questions/19560498/faster-way-to-remove-stop-words-in-python  
https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [10]:
%%time
#Removing stop words from the comments, where stop words are defined in NLTK stop words dictionary
from nltk.corpus import stopwords
stopWords = stopwords.words("english")

def removeStopWordsInComment(row):
    return ' '.join([word for word in row['comment_text'].split() if word not in stopWords])
    
comment_data['comment_no_stop_words'] = ''
comment_data['comment_no_stop_words'] = comment_data.apply(removeStopWordsInComment, axis=1)

/Users/gregdecanio/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


CPU times: user 1min 12s, sys: 1.01 s, total: 1min 13s
Wall time: 1min 16s


## Removing Punctuation
https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string  
According to the Stack Overflow, I could probably configure a more efficient way to remove punctuation. However, what I have currently works fine and is adequate.

In [11]:
%%time
#Removing puncuation from the comments, where punctuation is defined by the STRING punctuation dictionary
import string
punc = set(string.punctuation)

def removePunctuation(row):
    return ''.join([ch for ch in row['comment_no_stop_words'] if ch not in punc])

comment_data['comment_no_stop_punc'] = ''
comment_data['comment_no_stop_punc'] = comment_data.apply(removePunctuation, axis=1)

CPU times: user 9.58 s, sys: 217 ms, total: 9.8 s
Wall time: 9.96 s


In [12]:
comment_data[['comment_text', 'target', 'toxicity_classification','comment_no_stop_words', 'comment_no_stop_punc']]

,comment_text,target,toxicity_classification,comment_no_stop_words,comment_no_stop_punc
0,"What a great topic to focus on: ""how the Catho...",0.000000,not-toxic,"What great topic focus on: ""how Catholic churc...",What great topic focus on how Catholic church ...
1,There is so much of this program that is compl...,0.166667,not-toxic,There much program completely phony needs disc...,There much program completely phony needs disc...
2,"I'm sorry, but when you run as a Republican, s...",0.000000,not-toxic,"I'm sorry, run Republican, switch parties (and...",Im sorry run Republican switch parties and kee...
3,Torytwo. Your names says it all.,0.000000,not-toxic,Torytwo. Your names says all.,Torytwo Your names says all
4,I wouldn't brag about being a Bernie supporter...,0.000000,not-toxic,I brag Bernie supporter. He sang praises Venez...,I brag Bernie supporter He sang praises Venezu...
5,"""I would have thought the Koch brothers' embra...",0.000000,not-toxic,"""I would thought Koch brothers' embrace aborti...",I would thought Koch brothers embrace abortion...
6,Its painfully clear that Donald Trump's only c...,0.500000,toxic,Its painfully clear Donald Trump's capability ...,Its painfully clear Donald Trumps capability i...
7,"Metasphere writes:\n\n""Trudeau is a U.S. lapdo...",0.400000,not-toxic,"Metasphere writes: ""Trudeau U.S. lapdog, Freel...",Metasphere writes Trudeau US lapdog Freeland n...
8,Anti-Russian--you hypocrite!\nDo you call anyo...,0.738462,toxic,Anti-Russian--you hypocrite! Do call anyone di...,AntiRussianyou hypocrite Do call anyone disagr...
9,"Gee, Nigel, another notch on your belt. Thank...",0.000000,not-toxic,"Gee, Nigel, another notch belt. Thanks giving ...",Gee Nigel another notch belt Thanks giving us ...


# Splitting Data Into Test/Train Sets 

In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(comment_data, test_size=0.33, random_state=42)
print "Training data size: ", len(train)
print "Testing data size: ", len(test)

Training data size:  77050
Testing data size:  37950


In [14]:
train = train[['comment_text', 'target', 'toxicity_classification', 'comment_no_stop_punc']]
test = test[['comment_text', 'target', 'toxicity_classification', 'comment_no_stop_punc']]
train.head()

,comment_text,target,toxicity_classification,comment_no_stop_punc
25219,Your quote from the article really describes A...,0.0000,not-toxic,Your quote article really describes Arab Sprin...
97436,The Crooks always seems to con there way in an...,0.6500,toxic,The Crooks always seems con way win debate kan...
4346,Many illegal aliens are white hispanics.Many w...,0.5375,toxic,Many illegal aliens white hispanicsMany white ...
16948,Total misrepresentation. Why was Bernie agains...,0.0000,not-toxic,Total misrepresentation Why Bernie it What AFL...
89159,When was this announced?,0.0000,not-toxic,When announced


# TF-IDF

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_unclean_vect = CountVectorizer()
tfidf_unclean_transformer = TfidfTransformer()

count_clean_vect = CountVectorizer()
tfidf_clean_transformer = TfidfTransformer()

#Performing TF and TF-IDF transformation on training data

train_unclean_counts = count_unclean_vect.fit_transform(train['comment_text'])
print "Dimensions of Unclean Training Data Document Term Matrix: ", train_unclean_counts.shape
train_unclean_tfidf = tfidf_unclean_transformer.fit_transform(train_unclean_counts)
print "Dimensions of Unclean Training Data TF_IDF Matrix: ", train_unclean_tfidf.shape

train_clean_counts = count_clean_vect.fit_transform(train['comment_no_stop_punc'])
print "Dimensions of Clean Training Data Document Term Matrix: ", train_clean_counts.shape
train_clean_tfidf = tfidf_clean_transformer.fit_transform(train_clean_counts)
print "Dimensions of Clean Training Data TF_IDF Matrix: ", train_clean_tfidf.shape

Dimensions of Unclean Training Data Document Term Matrix:  (77050, 69165)
Dimensions of Unclean Training Data TF_IDF Matrix:  (77050, 69165)
Dimensions of Clean Training Data Document Term Matrix:  (77050, 90324)
Dimensions of Clean Training Data TF_IDF Matrix:  (77050, 90324)


In [16]:
#Performing TF and TF-IDF transformation on test data

test_unclean_counts = count_unclean_vect.transform(test['comment_text'])
print "Dimensions of Unclean Test Data Document Term Matrix: ", test_unclean_counts.shape
test_unclean_tfidf = tfidf_unclean_transformer.transform(test_unclean_counts)
print "Dimensions of Unclean Test Data TF_IDF Matrix: ", test_unclean_tfidf.shape

test_clean_counts = count_clean_vect.transform(test['comment_no_stop_punc'])
print "Dimensions of Clean Test Data Document Term Matrix: ", test_clean_counts.shape
test_clean_tfidf = tfidf_clean_transformer.transform(test_clean_counts)
print "Dimensions of Clean Test Data TF_IDF Matrix: ", test_clean_tfidf.shape

Dimensions of Unclean Test Data Document Term Matrix:  (37950, 69165)
Dimensions of Unclean Test Data TF_IDF Matrix:  (37950, 69165)
Dimensions of Clean Test Data Document Term Matrix:  (37950, 90324)
Dimensions of Clean Test Data TF_IDF Matrix:  (37950, 90324)


# Naive Bayes Classification
https://scikit-learn.org/0.19/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB

In [17]:
%%time
#Training and creating the Multinomial Naive Bayes model (unclean)
from sklearn.naive_bayes import MultinomialNB
modelNB_unclean = MultinomialNB().fit(train_unclean_tfidf, train['toxicity_classification'])

CPU times: user 275 ms, sys: 75.1 ms, total: 350 ms
Wall time: 553 ms


In [18]:
%%time
#Training and creating the Multinomial Naive Bayes model (clean)
modelNB_clean = MultinomialNB().fit(train_clean_tfidf, train['toxicity_classification'])

CPU times: user 283 ms, sys: 37.3 ms, total: 320 ms
Wall time: 352 ms


In [19]:
%%time
#Calulate accuracy of predictions
predsNB_unclean = modelNB_unclean.predict(test_unclean_tfidf)
accNB_unclean = np.mean(predsNB_unclean == test['toxicity_classification']) 
print "Accuracy of Naive Bayes Classifier (Unclean): ", accNB_unclean, "\n"

predsNB_clean = modelNB_clean.predict(test_clean_tfidf)
accNB_clean = np.mean(predsNB_clean == test['toxicity_classification']) 
print "Accuracy of Naive Bayes Classifier (Clean): ", accNB_clean, "\n"

Accuracy of Naive Bayes Classifier (Unclean):  0.845876152833 

Accuracy of Naive Bayes Classifier (Clean):  0.84534914361 

CPU times: user 56.1 ms, sys: 22.3 ms, total: 78.3 ms
Wall time: 96.4 ms


In [20]:
%%time
#Show confustion matrix of predictions
print(metrics.classification_report(test['toxicity_classification'], predsNB_unclean))
print "Confusion Matrix (Unclean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsNB_unclean)

print(metrics.classification_report(test['toxicity_classification'], predsNB_clean))
print "Confusion Matrix (Clean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsNB_clean)

             precision    recall  f1-score   support

  not-toxic       0.85      1.00      0.92     31988
      toxic       0.91      0.02      0.04      5962

avg / total       0.86      0.85      0.78     37950

Confusion Matrix (Unclean):
[[31975    13]
 [ 5836   126]]
             precision    recall  f1-score   support

  not-toxic       0.85      1.00      0.92     31988
      toxic       0.95      0.02      0.03      5962

avg / total       0.86      0.85      0.78     37950

Confusion Matrix (Clean):
[[31983     5]
 [ 5864    98]]
CPU times: user 742 ms, sys: 15 ms, total: 757 ms
Wall time: 788 ms


In [33]:
%%time
#Setting grid of hyper parameters to test
parameters = {'fit_prior' : (True, False),
              'alpha' : np.array([1,0.1,0.01,0.001,0.0001,0])}
#Performing hyper parameter fitting
gs_modelNB_clean = GridSearchCV(modelNB_clean, parameters, n_jobs=1)
gs_modelNB_clean = gs_modelNB_clean.fit(train_clean_tfidf, train['toxicity_classification'])

CPU times: user 14.8 s, sys: 914 ms, total: 15.7 s
Wall time: 16 s


In [34]:
%%time
#Seeing results of best hypertuned parameted
print gs_modelNB_clean
print "Highest accuracy of all models computed: ", gs_modelNB_clean.best_score_
print "Parameter values of highest accuracy model: ", gs_modelNB_clean.best_params_

GridSearchCV(cv=None, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e+00,   1.00000e-01,   1.00000e-02,   1.00000e-03,
         1.00000e-04,   0.00000e+00]), 'fit_prior': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
Highest accuracy of all models computed:  0.855626216742
Parameter values of highest accuracy model:  {'alpha': 0.10000000000000001, 'fit_prior': True}
CPU times: user 1.91 ms, sys: 93 µs, total: 2 ms
Wall time: 2.08 ms


In [35]:
gs_predsNB_clean = gs_modelNB_clean.predict(test_clean_tfidf)
#Classification metrics and confusion matrix of hyper tuned model
print(metrics.classification_report(test['toxicity_classification'], gs_predsNB_clean))
print "Confusion Matrix:\n", metrics.confusion_matrix(test['toxicity_classification'], gs_predsNB_clean)

             precision    recall  f1-score   support

  not-toxic       0.86      0.99      0.92     31988
      toxic       0.81      0.15      0.26      5962

avg / total       0.86      0.86      0.82     37950

Confusion Matrix:
[[31778   210]
 [ 5044   918]]


# KNN Classification
Currently # of neighbors is arbitraily set to 2 (Greg, 11/20/2019)  
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier  
https://scikit-learn.org/stable/modules/neighbors.html

In [ ]:
'''
%%time
#Training and creating the K Nearest Neighbors model (unclean)
from sklearn.neighbors import KNeighborsClassifier
modelKNN_unclean = KNeighborsClassifier(n_neighbors=2, algorithm='brute').fit(train_unclean_tfidf, train['toxicity_classification'])
'''

In [ ]:
'''
%%time
#Training and creating the K Nearest Neighbors model (clean)
from sklearn.neighbors import KNeighborsClassifier
modelKNN_clean = KNeighborsClassifier(n_neighbors=2, algorithm='brute').fit(train_clean_tfidf, train['toxicity_classification'])
'''

In [ ]:
'''
%%time
#Calulate accuracy of predictions
predsKNN_unclean = modelKNN_unclean.predict(test_unclean_tfidf)
accKNN_unclean = np.mean(predsKNN_unclean == test['toxicity_classification'])
print "Accuracy of K Nearest Neighbors Classifier (Unclean): ", accKNN_unclean

predsKNN_clean = modelKNN_unclean.predict(test_clean_tfidf)
accKNN_clean = np.mean(predsKNN_clean == test['toxicity_classification'])
print "Accuracy of K Nearest Neighbors Classifier (Unclean): ", accKNN_clean
'''

In [ ]:
'''
%%time
#Show confustion matrix of predictions
print(metrics.classification_report(test['toxicity_classification'], predsKNN_unclean))
print "Confusion Matrix (Unclean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsKNN_unclean)

print(metrics.classification_report(test['toxicity_classification'], predsKNN_clean))
print "Confusion Matrix (Clean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsKNN_clean)
'''

# SVC Classification
https://scikit-learn.org/stable/modules/svm.html#classification

In [ ]:
'''
%%time
#Training and creating the SVC model (unclean)
from sklearn import svm
modelSVC_unclean = svm.SVC(kernel='linear').fit(train_unclean_tfidf, train['toxicity_classification'])
'''

In [ ]:
'''
%%time
#Training and creating the SVC model (clean)
modelSVC_clean = svm.SVC(kernel='linear').fit(train_clean_tfidf, train['toxicity_classification'])
'''

In [ ]:
'''
%%time
#Calulate accuracy of predictions
predsSVC_unclean = modelSVC_unclean.predict(test_unclean_tfidf)
accSVC_unclean = np.mean(predsSVC_unclean == test['toxicity_classification'])
print "Accuracy of SCV Classifier (Unclean): ", accSVC_unclean

predsSVC_clean = modelSVC_clean.predict(test_clean_tfidf)
accSVC_clean = np.mean(predsSVC_clean == test['toxicity_classification'])
print "Accuracy of SVC Classifier (Clean): ", accSVC_clean
'''

In [ ]:
'''
%%time
#Show confustion matrix of predictions
print(metrics.classification_report(test['toxicity_classification'], predsSVC_unclean))
print "Confusion Matrix (Unclean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsSVC_unclean)

print(metrics.classification_report(test['toxicity_classification'], predsSVC_clean))
print "Confusion Matrix (Clean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsSVC_clean)
'''



# SGD Classification
https://scikit-learn.org/stable/modules/sgd.html#classification

In [36]:
%%time
#Training and creating the SGD Classifier model (unclean)
from sklearn.linear_model import SGDClassifier
modelSGD_unclean = SGDClassifier(loss='hinge', penalty='l2',
                           max_iter=5).fit(train_unclean_tfidf, train['toxicity_classification'])

CPU times: user 226 ms, sys: 73.1 ms, total: 299 ms
Wall time: 350 ms


In [37]:
%%time
#Training and creating the SGD Classifier model (clean)
modelSGD_clean = SGDClassifier(loss='hinge', penalty='l2',
                           max_iter=5).fit(train_clean_tfidf, train['toxicity_classification'])

CPU times: user 202 ms, sys: 36.3 ms, total: 239 ms
Wall time: 242 ms


In [38]:
%%time
#Calulate accuracy of predictions
predsSGD_unclean = modelSGD_unclean.predict(test_unclean_tfidf)
accSGD_unclean = np.mean(predsSGD_unclean == test['toxicity_classification']) 
print "Accuracy of SGD Classifier (Unclean): ", accSGD_unclean, "\n"

predsSGD_clean = modelSGD_clean.predict(test_clean_tfidf)
accSGD_clean = np.mean(predsSGD_clean == test['toxicity_classification']) 
print "Accuracy of SGD Classifier (Clean): ", accSGD_clean, "\n"

Accuracy of SGD Classifier (Unclean):  0.87512516469 

Accuracy of SGD Classifier (Clean):  0.876785243742 

CPU times: user 38.5 ms, sys: 50.3 ms, total: 88.7 ms
Wall time: 89.9 ms


In [39]:
%%time
#Show confustion matrix of predictions
print(metrics.classification_report(test['toxicity_classification'], predsSGD_unclean))
print "Confusion Matrix (Unclean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsSGD_unclean)

print(metrics.classification_report(test['toxicity_classification'], predsSGD_clean))
print "Confusion Matrix (Clean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsSGD_clean)

             precision    recall  f1-score   support

  not-toxic       0.87      1.00      0.93     31988
      toxic       0.93      0.22      0.36      5962

avg / total       0.88      0.88      0.84     37950

Confusion Matrix (Unclean):
[[31889    99]
 [ 4640  1322]]
             precision    recall  f1-score   support

  not-toxic       0.87      1.00      0.93     31988
      toxic       0.92      0.24      0.38      5962

avg / total       0.88      0.88      0.84     37950

Confusion Matrix (Clean):
[[31869   119]
 [ 4557  1405]]
CPU times: user 771 ms, sys: 14.3 ms, total: 785 ms
Wall time: 802 ms


In [41]:
%%time
#Setting grid of hyper parameters to test
parameters = {'loss' : ('hinge', 'modified_huber'),
              'penalty' : ('l2', 'l1'),
              'max_iter' : np.array([1,3,5,7,10])}
#Performing hyper parameter fitting
gs_modelSGD_clean = GridSearchCV(modelSGD_clean, parameters, n_jobs=1)
gs_modelSGD_clean = gs_modelSGD_clean.fit(train_clean_tfidf, train['toxicity_classification'])

CPU times: user 24.1 s, sys: 1.66 s, total: 25.7 s
Wall time: 26.3 s


In [42]:
%%time
#Seeing results of best hypertuned parameted
print gs_modelSGD_clean
print "Highest accuracy of all models computed: ", gs_modelSGD_clean.best_score_
print "Parameter values of highest accuracy model: ", gs_modelSGD_clean.best_params_

GridSearchCV(cv=None, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ('l2', 'l1'), 'loss': ('hinge', 'modified_huber'), 'max_iter': array([ 1,  3,  5,  7, 10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
Highest accuracy of all models computed:  0.900674886437
Parameter values of highest accuracy model:  {'penalty': 'l1', 'loss': 'modified_huber', 'max_iter': 10}
CPU times: user 3.39 ms, sys: 578 µs, total: 3.97 ms
Wall time: 6.59 ms


In [43]:
gs_predsSGD_clean = gs_modelSGD_clean.predict(test_clean_tfidf)
#Classification metrics and confusion matrix of hyper tuned model
print(metrics.classification_report(test['toxicity_classification'], gs_predsSGD_clean))
print "Confusion Matrix:\n", metrics.confusion_matrix(test['toxicity_classification'], gs_predsSGD_clean)

             precision    recall  f1-score   support

  not-toxic       0.91      0.98      0.94     31988
      toxic       0.81      0.50      0.61      5962

avg / total       0.90      0.90      0.89     37950

Confusion Matrix:
[[31271   717]
 [ 3000  2962]]


# Random Forest Classification
https://scikit-learn.org/0.19/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier

In [47]:
%%time
#Training the data and creating the Random Forest model (unclean)
from sklearn.ensemble import RandomForestClassifier
modelRF_unclean = RandomForestClassifier().fit(train_unclean_tfidf, train['toxicity_classification'])

CPU times: user 37.1 s, sys: 584 ms, total: 37.7 s
Wall time: 41 s


In [48]:
%%time
#Training the data and creating the Random Forest model (clean)
modelRF_clean = RandomForestClassifier().fit(train_clean_tfidf, train['toxicity_classification'])

CPU times: user 49.1 s, sys: 650 ms, total: 49.7 s
Wall time: 54.4 s


In [49]:
%%time
#Calulate accuracy of predictions
predsRF_unclean = modelRF_unclean.predict(test_unclean_tfidf)
accRF_unclean = np.mean(predsRF_unclean == test['toxicity_classification']) 
print "Accuracy of RF Classifier (Unclean): ", accRF_unclean, "\n"

predsRF_clean = modelRF_clean.predict(test_clean_tfidf)
accRF_clean = np.mean(predsRF_clean == test['toxicity_classification']) 
print "Accuracy of RF Classifier (Clean): ", accRF_clean, "\n"

Accuracy of RF Classifier (Unclean):  0.880553359684 

Accuracy of RF Classifier (Clean):  0.887325428195 

CPU times: user 4.59 s, sys: 160 ms, total: 4.75 s
Wall time: 5.26 s


In [50]:
%%time
#Show confustion matrix of predictions
print(metrics.classification_report(test['toxicity_classification'], predsRF_unclean))
print "Confusion Matrix (Unclean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsRF_unclean)

print(metrics.classification_report(test['toxicity_classification'], predsRF_clean))
print "Confusion Matrix (Clean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsRF_clean)

             precision    recall  f1-score   support

  not-toxic       0.88      0.99      0.93     31988
      toxic       0.85      0.29      0.43      5962

avg / total       0.88      0.88      0.85     37950

Confusion Matrix (Unclean):
[[31690   298]
 [ 4235  1727]]
             precision    recall  f1-score   support

  not-toxic       0.90      0.98      0.94     31988
      toxic       0.79      0.39      0.52      5962

avg / total       0.88      0.89      0.87     37950

Confusion Matrix (Clean):
[[31353   635]
 [ 3641  2321]]
CPU times: user 1 s, sys: 23.6 ms, total: 1.03 s
Wall time: 1.11 s


In [53]:
%%time
#Setting grid of hyper parameters to test
parameters = {'n_estimators' : np.array([5,10,20]),
              'criterion' : ('gini', 'entropy'),
              'max_depth' : (None, 10, 20),
              'max_features' : ('auto', 'log2')}
#Performing hyper parameter fitting
gs_modelRF_clean = GridSearchCV(modelRF_clean, parameters, n_jobs=1)
gs_modelRF_clean = gs_modelRF_clean.fit(train_clean_tfidf, train['toxicity_classification'])

CPU times: user 1h 7min 27s, sys: 32.8 s, total: 1h 8min
Wall time: 1h 12min 43s


In [54]:
%%time
#Seeing results of best hypertuned parameted
print gs_modelRF_clean
print "Highest accuracy of all models computed: ", gs_modelRF_clean.best_score_
print "Parameter values of highest accuracy model: ", gs_modelRF_clean.best_params_

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([ 5, 10, 20]), 'max_features': ('auto', 'log2'), 'criterion': ('gini', 'entropy'), 'max_depth': (None, 10, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
Highest accuracy of all models computed:  0.890759247242
Parameter values of highest accuracy model:  {'max_features': 'auto', 'n_estimators': 20, 'criterion': 'gini', 'max_depth': None}
CPU times: user 3.04 ms, sys: 

In [55]:
gs_predsRF_clean = gs_modelRF_clean.predict(test_clean_tfidf)
#Classification metrics and confusion matrix of hyper tuned model
print(metrics.classification_report(test['toxicity_classification'], gs_predsRF_clean))
print "Confusion Matrix:\n", metrics.confusion_matrix(test['toxicity_classification'], gs_predsRF_clean)

             precision    recall  f1-score   support

  not-toxic       0.90      0.98      0.94     31988
      toxic       0.80      0.41      0.55      5962

avg / total       0.88      0.89      0.88     37950

Confusion Matrix:
[[31372   616]
 [ 3496  2466]]


# Decision Tree Classification
https://scikit-learn.org/0.19/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [23]:
%%time
#Training the data and creating the Decision Tree model (unclean)
from sklearn.tree import DecisionTreeClassifier
modelDT_unclean = DecisionTreeClassifier().fit(train_unclean_tfidf, train['toxicity_classification'])

CPU times: user 6min 17s, sys: 2.01 s, total: 6min 19s
Wall time: 6min 32s


In [25]:
%%time
#Training the data and creating the Decision Tree model (clean)
modelDT_clean = DecisionTreeClassifier().fit(train_clean_tfidf, train['toxicity_classification'])

CPU times: user 5min 50s, sys: 619 ms, total: 5min 50s
Wall time: 5min 51s


In [26]:
%%time
#Calulate accuracy of predictions
predsDT_unclean = modelDT_unclean.predict(test_unclean_tfidf)
accDT_unclean = np.mean(predsDT_unclean == test['toxicity_classification']) 
print "Accuracy of DT Classifier (Unclean): ", accDT_unclean, "\n"

predsDT_clean = modelDT_clean.predict(test_clean_tfidf)
accDT_clean = np.mean(predsDT_clean == test['toxicity_classification']) 
print "Accuracy of DT Classifier (Clean): ", accDT_clean, "\n"

Accuracy of DT Classifier (Unclean):  0.87093544137 

Accuracy of DT Classifier (Clean):  0.872516469038 

CPU times: user 625 ms, sys: 59.4 ms, total: 684 ms
Wall time: 690 ms


In [27]:
%%time
#Show confustion matrix of predictions
print(metrics.classification_report(test['toxicity_classification'], predsDT_unclean))
print "Confusion Matrix (Unclean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsDT_unclean)

print(metrics.classification_report(test['toxicity_classification'], predsDT_clean))
print "Confusion Matrix (Clean):\n", metrics.confusion_matrix(test['toxicity_classification'], predsDT_clean)

             precision    recall  f1-score   support

  not-toxic       0.92      0.93      0.92     31988
      toxic       0.59      0.57      0.58      5962

avg / total       0.87      0.87      0.87     37950

Confusion Matrix (Unclean):
[[29677  2311]
 [ 2587  3375]]
             precision    recall  f1-score   support

  not-toxic       0.92      0.93      0.92     31988
      toxic       0.60      0.59      0.59      5962

avg / total       0.87      0.87      0.87     37950

Confusion Matrix (Clean):
[[29620  2368]
 [ 2470  3492]]
CPU times: user 908 ms, sys: 16.3 ms, total: 924 ms
Wall time: 924 ms


## How a Pipeline Is Setup

In [ ]:
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('model', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)),
 ])